# SET UP

In this notebook, the results of differing sizes of the dataset can be measured through sampling the dataset and then running the various remedy methods and identification methods to determine the overall time that these methods take. 

The notebook is divided into the setup where data is read in (and change the number of records to sample), contains the helper functions (for more details, see demo file and refer to paper), and the cells to measure the time for the remedy and identification methods. 

For more details, refer to the paper.


## Imports and Dataset processing

In [ ]:
import pandas as pd
import time
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
import copy
from sympy import Symbol
from sympy.solvers import solve
pd.options.mode.chained_assignment = None 

In [ ]:
url = "https://raw.githubusercontent.com/niceIrene/remedy/main/datasets/CleanAdult_numerical_cat.csv"
data = pd.read_csv(url)

data

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,2,226802,1,7,4,6,3,2,1,0,0,0,38,0
1,1,2,89814,11,9,2,4,0,4,1,0,0,1,38,0
2,0,1,336951,7,12,2,10,0,4,1,0,0,0,38,1
3,1,2,160323,15,10,2,6,0,2,1,1,0,0,38,1
4,1,2,198693,0,6,4,7,1,4,1,0,0,0,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,0,2,257302,7,12,2,12,5,4,0,0,0,0,38,0
45218,1,2,154374,11,9,2,6,0,4,1,0,0,0,38,1
45219,2,2,151910,11,9,6,0,4,4,0,0,0,0,38,0
45220,0,2,201490,11,9,4,0,3,4,1,0,0,0,38,0


In [ ]:
# change the value here for the number of records
value = 6000
data = data.sample(value)

#for 48K records
# data2 = data.sample(3000)
# data = pd.concat([data, data2], ignore_index=True)
data

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,2,226802,1,7,4,6,3,2,1,0,0,0,38,0
1,1,2,89814,11,9,2,4,0,4,1,0,0,1,38,0
2,0,1,336951,7,12,2,10,0,4,1,0,0,0,38,1
3,1,2,160323,15,10,2,6,0,2,1,1,0,0,38,1
4,1,2,198693,0,6,4,7,1,4,1,0,0,0,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48217,1,2,145636,11,9,2,3,0,4,1,1,0,1,38,1
48218,2,2,155233,15,10,0,0,4,4,0,0,0,0,38,0
48219,0,2,123335,0,6,4,11,3,4,0,0,0,0,38,0
48220,0,2,378322,11,9,4,2,1,4,1,0,1,1,38,0


In [ ]:
# get training and testing set



columns_all = ['age', 'workclass', 'education', 'educational-num', 'marital-status', 
                                            'occupation', 'relationship', 'race','gender', 'capital-gain', 'capital-loss',
                                            'hours-per-week', 'native-country']
columns_compas = ['age', 'education',  'marital-status', 'occupation', 
                  'relationship', 'race','gender', 'native-country']
# compas_y = 'class'
compas_y = 'income'
def split_train_test(data,test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

def get_train_test(data, split, list_cols, y_label):
  all_list = copy.deepcopy(list_cols)
  all_list.append(y_label)
  data = pd.DataFrame(data, columns = all_list)
  train_set,test_set = split_train_test(data,split)
  print(len(train_set), "train +", len(test_set), "test")
  train_x = pd.DataFrame(train_set, columns = list_cols)
  train_label = train_set[y_label]
  test_x = pd.DataFrame(test_set, columns = list_cols)
  test_label = test_set[y_label]
  return train_x, test_x, train_label, test_label, train_set, test_set

In [ ]:
train_x, test_x, train_label, test_label, train_set, test_set  = get_train_test(data, 0.3, columns_all, compas_y)

###################

### about decision tree
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

param = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 20, 30, 40, 50, 100], 'random_state':[17]}

grid = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
grid.fit(train_x, train_label)

data_all = pd.concat([train_x,test_x])
data_predict = grid.predict(data_all)
test_predict = grid.predict(test_x)
data['predicted'] = data_predict

test_set['predicted'] = test_predict
print(test_set)
print(test_x)


33756 train + 14466 test
       age  workclass  education  educational-num  marital-status  occupation  \
8206     1          2         12               14               0           3   
18067    2          0         11                9               2           2   
47552    0          2         11                9               4          10   
33923    2          2         11                9               4           6   
37540    1          2         11                9               2           6   
...    ...        ...        ...              ...             ...         ...   
41179    1          4          9               13               4          12   
18233    1          1          9               13               0           9   
9029     1          4         11                9               0           2   
48212    1          2          9               13               0           5   
36804    1          4         12               14               2          11   

  

In [ ]:
def fpr_onegroup(true, predict):
    fp = 0
    tn = 0
    for i in range(len(true)):
        if (true[i] == 0 and predict[i] == 1):
            fp += 1 
        if(true[i] == 0 and predict[i] == 0):
            tn += 1
    return fp/(fp+tn)


def fnr_onegroup(true, predict):
    fn = 0
    tp = 0
    for i in range(len(true)):
        if (true[i] == 1 and predict[i] == 0):
            fn += 1 
        if(true[i] == 1 and predict[i] == 1):
            tp += 1
    return fn/(fn+tp)

In [ ]:
fpr = fpr_onegroup(list(test_label), test_predict)
print("fpr is " , fpr)

fnr = fnr_onegroup(list(test_label), test_predict)
print("fnr is " , fnr)

fpr is  0.06504290063659009
fnr is  0.4910394265232975


In [ ]:
accuracy = accuracy_score(test_label, test_predict)
print("accuracy is " , accuracy)


accuracy is  0.8281487626157887


## For entire dataset


In [ ]:
import itertools
def get_unfair_group(list_parse, entire = 1):
  unfair_group = []
  unfair_dict = {}
  names = []
  for col in columns_compas:
    found = False
    for item in list_parse:
      attr_given = item.split("=")[0]
      if col == attr_given:
        unfair_group.append(int(item.split("=")[1]))
        names.append(attr_given)
        unfair_dict[attr_given] = int(item.split("=")[1])
        found = True
  # if use the entire dataset
  if entire:
    return unfair_group, names, columns_compas, unfair_dict

  return unfair_group, names, list(set(columns_compas).symmetric_difference(set(names))), unfair_dict
def candidate_groups(skew_candidates, unfair_dict, ordering, names):
  candidate_combos = []
  candidate_ind = {}
  num = 0
  for i in range(len(skew_candidates)+1):
    temp_candidate = list(itertools.combinations(skew_candidates, i))
    for tc in temp_candidate:
      candidate_ind[num] = list(tc)
      num += 1
  return candidate_ind

def name_val_dict(train_set,names):
  names_values = {}
  for n in names:
    names_values[n] = list(train_set[n].unique())
  return names_values

  

In [ ]:
def get_temp(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp2 = temp.groupby(names2)['cnt'].count().reset_index()
  temp2['cnt'].sum()
  return temp2, names
temp2, names = get_temp(train_set, columns_compas, compas_y)
temp2

,age,education,marital-status,occupation,relationship,race,gender,native-country,income,cnt
0,0,0,0,2,3,4,1,38,0,1
1,0,0,0,5,1,4,0,38,0,1
2,0,0,0,5,1,4,1,38,0,1
3,0,0,0,5,3,4,1,38,0,1
4,0,0,0,5,4,4,0,38,0,2
...,...,...,...,...,...,...,...,...,...,...
8734,2,15,6,11,4,4,0,38,1,1
8735,2,15,6,12,1,4,0,38,0,4
8736,2,15,6,12,4,4,0,38,0,2
8737,2,15,6,13,1,4,0,38,0,1


In [ ]:
def get_temp_g(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp_g = temp.groupby(names)['cnt'].count().reset_index()
  return temp, temp_g

In [ ]:
unfair_group, unfair_names, skew_candidates, unfair_dict = get_unfair_group([])
print(unfair_group, unfair_names, skew_candidates, unfair_dict)
all_names = candidate_groups(skew_candidates, unfair_dict, columns_compas, unfair_names)
names_values = name_val_dict(train_set, names)
print(all_names)
print(names_values)
all_names_lst = list(all_names.keys())[len(columns_compas)+1:]
all_names_lst.reverse()
# all_names_lst

[] [] ['age', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country'] {}
{0: [], 1: ['age'], 2: ['education'], 3: ['marital-status'], 4: ['occupation'], 5: ['relationship'], 6: ['race'], 7: ['gender'], 8: ['native-country'], 9: ['age', 'education'], 10: ['age', 'marital-status'], 11: ['age', 'occupation'], 12: ['age', 'relationship'], 13: ['age', 'race'], 14: ['age', 'gender'], 15: ['age', 'native-country'], 16: ['education', 'marital-status'], 17: ['education', 'occupation'], 18: ['education', 'relationship'], 19: ['education', 'race'], 20: ['education', 'gender'], 21: ['education', 'native-country'], 22: ['marital-status', 'occupation'], 23: ['marital-status', 'relationship'], 24: ['marital-status', 'race'], 25: ['marital-status', 'gender'], 26: ['marital-status', 'native-country'], 27: ['occupation', 'relationship'], 28: ['occupation', 'race'], 29: ['occupation', 'gender'], 30: ['occupation', 'native-country'], 31: ['relationship', 'race'], 3

## Helper Functions

### General Helper Functions


In [ ]:
def get_one_degree_neighbors(temp2, names, group_lst):
    result = []
    for i in range(len(group_lst)):
        d = copy.copy(temp2)
        for k in range(len(group_lst)):
            if k != i:
                d = d[d[names[k]] == group_lst[k]]
            else:
                d = d[d[names[k]] != group_lst[k]]
        
        result.append(d)
    return result

In [ ]:
# compute the pos/neg ration of this neighbor
def compute_neighbors(group_lst, result):
    # compute the ratio of positive and negative records
    start2 = time.time()
    pos = 0
    neg = 0 
    for r in result:
        total  = r['cnt'].sum()
        r = r[r[compas_y] == 1]
        pos += r['cnt'].sum()
        neg += total - r['cnt'].sum()
    if(neg == 0):
        return (pos, neg, -1)
    end2 = time.time()
    
    return(pos, neg, pos/neg)

In [ ]:
def compute_diff_add_and_remove(group_lst, temp2, need_positive_or_negative, label, names):
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        d = d[d[names[i]] == group_lst[i]]
        
    total =  d['cnt'].sum()
    # Total here was 0: here, errors when this is commented out
    if total == 0:
      return -1
    d = d[d[label] == 1]
    pos = d['cnt'].sum()
   
    neg = total - pos
    result = get_one_degree_neighbors(temp2,names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos
        x = Symbol('x')
      
        try:
          diff = solve((pos + x)/ (neg - x) - neighbors[2])[0]
        except:
          return -1
            
    else:
        #need negative
        x = Symbol('x')
        try:
          diff = solve((pos - x)/ (neg + x) - neighbors[2])[0]
        except:
          return -1
   
    return diff

In [ ]:
def compute_diff_add(group_lst, temp2, names, label_y, need_positive_or_negative):

    d = copy.copy(temp2)
    
    for i in range(len(group_lst)):

        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label_y] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    result = get_one_degree_neighbors(temp2, names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos

        x = Symbol('x')
        try:
          diff = solve((pos + x)/ neg -  neighbors[2])[0]
        except:
          return -1
      
    else:
        #need negative
      
        x = Symbol('x')
        try:
          diff = solve(pos/ (neg + x) -  neighbors[2])[0]
        except:
          return -1
    return diff

def compute_diff_remove(group_lst, temp2, names, label_y, need_positive_or_negative):
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
      
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label_y] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    result = get_one_degree_neighbors(temp2, names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos, remove some neg
        x = Symbol('x')
        try:
          diff = solve( pos/ (neg - x) -  neighbors[2])[0]
        except:
          return -1
       
    else:
        #need negative
        x = Symbol('x')
        try:
          diff = solve((pos -x )/ neg -  neighbors[2])[0]
        except:
          return -1 
      
    return diff


### Optimized Helper Function


In [ ]:
# helper function for optimized
def compute_neighbors_opt(group_lst,lst_of_counts, pos, neg):

    times = len(group_lst)
    pos_cnt = 0
    neg_cnt = 0
    for i in range(times):
        df_groupby = lst_of_counts[i]
        temp_group_lst_pos = copy.copy(group_lst)
        temp_group_lst_neg = copy.copy(group_lst)
        del temp_group_lst_pos[i]
        del temp_group_lst_neg[i]
        # count positive
        temp_group_lst_pos.append(1)
        group_tuple_pos = tuple(temp_group_lst_pos)
        if group_tuple_pos in df_groupby.keys():
            pos_cnt += df_groupby[group_tuple_pos]
        else:
            pos_cnt += 0
        # count negative
        temp_group_lst_neg.append(0)
        group_tuple_neg = tuple(temp_group_lst_neg)
        if group_tuple_neg in df_groupby.keys():
            neg_cnt += df_groupby[group_tuple_neg]
        else:
            neg_cnt += 0
    pos_val = pos_cnt - times* pos
    neg_val = neg_cnt - times* neg
   

    if neg_val == -1 or (neg_val == 0 and pos_val == 0):
        return (pos_val, neg_val, -1)
    if pos_val == 0 or neg_val == 0:
        return (pos_val, neg_val, 0)


    return (pos_val, neg_val, pos_val/neg_val)

In [ ]:
# get the list of neighbors
def get_one_degree_neighbors_opt(group_lst):
    start1 = time.time()
    result = []
    for i in range(len(group_lst)):
        d = copy.copy(group_lst)
        d[i] = 'x'
        result.append(d)
    end1 = time.time()
    return result

In [ ]:
def determine_problematic_opt(group_lst, names, temp2, lst_of_counts, label, threshold= 0.3):
    #0: ok group, 1: need negative records, 2: need positive records
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    neighbors = compute_neighbors_opt(group_lst,lst_of_counts, pos, neg)
    if(neighbors[2] == -1):
        # there is no neighbors
        return 0
    if(total > 30):
        # need to be large enough, need to adjust with different datasets.
        if neg == 0:
            if (pos > neighbors[2]):
                return 1
            if(pos <= neighbors[2]):
                return 0
        if (pos/(neg) - neighbors[2] > threshold):
            # too many positive records
            return 1
        if (neighbors[2] - pos/(neg) > threshold):
            return 2
    return 0

In [ ]:
def compute_problematic_opt(temp2, temp_g, names, label, lst_of_counts):
    need_pos = []
    need_neg = []
    for index, row in temp_g.iterrows():
        group_lst = []
        for n in names:
            group_lst.append(row[n])
        problematic = determine_problematic_opt(group_lst, names, temp2, lst_of_counts,label)

        if(problematic == 1):
            if group_lst not in need_neg:
                need_neg.append(group_lst)
        if(problematic == 2):
            if group_lst not in need_pos:
                need_pos.append(group_lst)
    return need_pos, need_neg

In [ ]:
# build the list of X00
def compute_lst_of_counts(temp, names, label):
    # get the list of group-by attributes
    lst_of_counts = []
    for i in range(len(names)):
        grp_names = copy.copy(names)
        del grp_names[i]
        grp_names.append(label)
        temp_df = temp.groupby(grp_names)['cnt'].count()
        lst_of_counts.append(temp_df)
    return lst_of_counts
    
def get_tuple(group_lst):
    return tuple(group_lst) 


def get_temp_g(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp_g = temp.groupby(names)['cnt'].count().reset_index()
  return temp, temp_g

### Remedy Algorithms

In [ ]:
from sklearn.naive_bayes import MultinomialNB
def pref_sampling_opt(train_set, cols_given, label, need_pos, need_neg):
    if len(need_pos)+ len(need_neg) > 0:
        temp_train_x = pd.DataFrame(train_set, columns = columns_all)
        temp_train_label = pd.DataFrame(train_set, columns = [label])
        temp_train_label = temp_train_label[label]
        temp_train_label = temp_train_label.astype('int')
        mnb = MultinomialNB()
        mnb = mnb.fit(temp_train_x, temp_train_label)
        probs = mnb.predict_proba(temp_train_x)[:,0]
        train_set["prob"] = abs(probs - 0.5)
    new_train_set = pd.DataFrame(columns = list(train_set.columns))
    updated_pos = 0
    for i in need_pos:
        # needs to updated more positive records
        
        temp_df = copy.deepcopy(train_set)
        for n in range(len(i)):
          temp_df = temp_df[temp_df[cols_given[n]] == i[n]]
        # update the skew and diff
        idx = list(temp_df.index)
        train_set.loc[idx, 'skewed'] = 1
        idx_pos = list(temp_df[(getattr(temp_df, label) == 1)].index)
        if(len(idx_pos) == 0):
          # if there is no positive
          idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
          neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          continue
        idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
        pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
        neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
        diff = compute_diff_add_and_remove(i, temp2,  1, compas_y, names)
        if diff == -1:
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          continue
        train_set.loc[idx, 'diff'] = int(diff)
        cnt = int(train_set.loc[idx_pos[0]]["diff"])
        updated_pos += cnt * 2 
        # add more records when there are not enough available records
        new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
        temp_cnt = cnt
        if len(pos_ranked) >= temp_cnt:
            new_train_set = pd.concat([new_train_set,pos_ranked[0:cnt]], ignore_index=True)
        else:
            while(temp_cnt > 0 ):
                new_train_set = pd.concat([new_train_set,pos_ranked[0:temp_cnt]], ignore_index=True) 
            # duplicate the dataframe
                temp_cnt = temp_cnt - len(pos_ranked)
        # duplicate the top cnt records from the pos
        # remove the top cnt records from the neg
        if cnt == 0:
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
        else:
          new_train_set = pd.concat([new_train_set, neg_ranked[cnt-1:-1]], ignore_index=True)
    updated_neg = 0
    # adding more records to the need_neg set
    for i in need_neg:
        # list of idx belongs to this group
        temp_df = copy.deepcopy(train_set)
        for n in range(len(i)):
          temp_df = temp_df[temp_df[cols_given[n]] == i[n]]
        # update the skew and diff
        idx = list(temp_df.index)
        train_set.loc[idx, 'skewed'] = 1
        idx_pos = list(temp_df[(getattr(temp_df, label) == 1)].index)
        idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
        if(len(idx_neg) == 0):
          pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          continue
        pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
        neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
        diff = compute_diff_add_and_remove(i, temp2, 0, compas_y, names)
        if diff == -1:
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          continue
        train_set.loc[idx, 'diff'] = int(diff)
        cnt = int(train_set.loc[idx_pos[0]]["diff"])
        updated_neg += cnt * 2 
        # add more records when there are not enough available records
        new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
        temp_cnt = cnt
        if len(neg_ranked) >= temp_cnt:
            new_train_set = pd.concat([new_train_set,neg_ranked[0:cnt]], ignore_index=True)
        else:
            while(temp_cnt > 0 ):
                new_train_set = pd.concat([new_train_set,neg_ranked[0:temp_cnt]], ignore_index=True) 
            # duplicate the dataframe
                temp_cnt = temp_cnt - len(neg_ranked)
        # duplicate the top cnt records from the pos
        # remove the top cnt records from the neg
        if cnt ==0:
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
        
        else:
          new_train_set = pd.concat([new_train_set, pos_ranked[cnt-1:-1]], ignore_index=True)

    # add the other irrelavant items:
    idx_irr = list(train_set[train_set['skewed'] == 0].index)
    irr_df = train_set.loc[idx_irr]
    new_train_set = pd.concat([new_train_set, irr_df], ignore_index=True)

    new_train_set.reset_index()
    return new_train_set

In [ ]:
def round_int(x):
    if x in [float("-inf"),float("inf")]: return 0
    return int(round(x))


def make_remove(d, group_lst, diff, names, label_y, need_positive_or_negative):
    temp = copy.deepcopy(d)
    for i in range(len(group_lst)):
        att_name = names[i]
        temp = temp[(temp[att_name] == group_lst[i])]
    temp = temp[(temp[label_y] == need_positive_or_negative)]
    # randomly generated diff samples
        #generated = temp
        # the number needed is more than the not needed numbers.
    if(diff>len(temp)):
        diff = len(temp)
    generated = temp.sample(n = diff, replace = False, axis = 0)
    return generated.index


def naive_downsampling(d, temp2, names, need_pos, need_neg, label_y):
    # add more records for all groups
    # The smote algorithm to boost the coverage
    for r in need_pos:
    # add more positive records
        # determine how many points to add
        diff = compute_diff_remove(r, temp2, names, label_y, need_positive_or_negative = 1)
        if diff == -1:
          continue
        diff = round_int(diff)
        # add more records
        samples_to_remove = make_remove(d, r, diff, names, label_y, need_positive_or_negative = 0)
        d.drop(index  = samples_to_remove, inplace = True)
    for k in need_neg:
        diff = compute_diff_remove(k, temp2, names, label_y, need_positive_or_negative = 0)
        if diff == -1:
          continue
        diff = round_int(diff)
        samples_to_remove = make_remove(d, k, diff, names, label_y, need_positive_or_negative = 1)
        d.drop(index  = samples_to_remove, inplace = True)
    return d

In [ ]:
from sklearn.naive_bayes import MultinomialNB
def round_int(x):
    if x in [float("-inf"),float("inf")]: return 0
    return int(round(x))

def get_depromotion(d, diff, group_lst, names, label_y, flag_depro):
    input_test = pd.DataFrame(d, columns = columns_compas)
    clf = MultinomialNB()
    temp_train_label = pd.DataFrame(d, columns = [label_y])
    temp_train_label = temp_train_label[label_y]
    temp_train_label = temp_train_label.astype('int')
    clf = clf.fit(input_test, temp_train_label)
    prob  = clf.predict_proba(input_test)[:,0]
    select = copy.deepcopy(d)
    select['prob'] = prob # the higher the probablity is, the more likely for it to be 0
    # filter out those belongs to this group
    for i in range(len(group_lst)):
        att_name = names[i]
        select = select[(select[att_name] == group_lst[i])]
    select = select[(select[label_y] == flag_depro)]
    # rank them according to the probability
    # filp the records and remove the records from d
    if (flag_depro == 0):
        select.sort_values(by="prob", ascending=True, inplace=True)
        select[label_y] = 1
    else:
        select.sort_values(by="prob", ascending=False, inplace=True)
        select[label_y] = 0
    head = select.head(diff)
    index_list = []
    index_list = list(head.index)
    d.drop(index_list,inplace = True)
    head.drop(columns = ['prob'],inplace = True)
    return head



def naive_massaging(d, temp2, names, need_pos, need_neg,label_y):
    # add more records for all groups
    # The smote algorithm to boost the coverage
    for r in need_pos:
        # print("adding more positive")
    # add more positive records
        # determine how many points to add
        diff = compute_diff_add_and_remove(r, temp2, 1, label_y, names)
        if diff == -1:
          continue
        diff =  round_int(diff)
        # add more records
        #0 for promotion
        samples_to_add = get_depromotion(d, diff, r, names, label_y, flag_depro = 0)
        d = pd.concat([d, samples_to_add])
    for k in need_neg:
        diff = compute_diff_add_and_remove(k, temp2, 0, label_y, names)
        if diff == -1:
          continue
        diff =  round_int(diff)
        #1 for demotion
        samples_to_add = get_depromotion(d, diff, k, names, label_y, flag_depro = 1)
        d = pd.concat([d, samples_to_add])
    return d

# OPTIMIZED ALGORITHM + REMEDY METHODS


In [ ]:
def candidate_groups(skew_candidates):
  candidate_combos = []
  candidate_ind = {}
  num = 0
  for i in range(len(skew_candidates)+1):
    temp_candidate = list(itertools.combinations(skew_candidates, i))
    for tc in temp_candidate:
      candidate_ind[num] = list(tc)
      num += 1
  return candidate_ind

In [ ]:
filter_count = 30

In [ ]:
import random
new_train_data = copy.deepcopy(train_set)
import csv

with open('adult_records.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    minus = 0
    all_names = candidate_groups(columns_compas[:len(columns_compas)- minus])
    all_names_lst = list(all_names.keys())[len(columns_compas)-minus+1:]
    all_names_lst.reverse()
    list_write = []
    list_write.append("Adult")
    list_write.append("Pref_opt")
    list_write.append(data.shape[0])
    list_write.append(train_set.shape[0])
    list_write.append(columns_compas[:len(columns_compas)-minus])
    list_write.append(len(columns_compas[:len(columns_compas)-minus]))
    excute_time = 0
    excute_time1 = 0
    print("started pref sampling")
    for a in all_names_lst:
        temp2, names = get_temp(new_train_data, all_names[a], compas_y)
        temp, temp_g = get_temp_g(new_train_data, names, compas_y)
        temp_g = temp_g[temp_g['cnt'] > filter_count]
        lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
        start = time.time()    
        need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
        end = time.time()
        excute_time += end - start

        new_train_data['skewed'] = 0
        new_train_data["diff"] = 0
        
        start1 = time.time()    
        new_train_data = pref_sampling_opt(new_train_data, names, compas_y, need_pos, need_neg)
        end1 = time.time()
        excute_time1 += end1 - start1   
    list_write.append(excute_time)
    list_write.append(excute_time1)
    writer.writerow(list_write)

    print("end pref sampling")

    new_train_data = copy.deepcopy(train_set)
    list_write = []
    list_write.append("Adult")
    list_write.append("Downsampling_opt")
    list_write.append(data.shape[0])
    list_write.append(train_set.shape[0])
    list_write.append(columns_compas[:len(columns_compas)-minus])
    list_write.append(len(columns_compas[:len(columns_compas)-minus]))
    excute_time = 0
    excute_time1 = 0
    print("start down sampling")
    for a in all_names_lst:
      temp2, names = get_temp(new_train_data, all_names[a], compas_y)
      temp, temp_g = get_temp_g(new_train_data, names, compas_y)
      temp_g = temp_g[temp_g['cnt'] > filter_count]
      lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
      start = time.time()    
      need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
      end = time.time()
      excute_time += end - start
      new_train_data['skewed'] = 0
      new_train_data["diff"] = 0

      start1 = time.time()
      new_train_data = naive_downsampling(new_train_data, temp2, names, need_pos, need_neg, compas_y)
      end1 = time.time()
      excute_time1 += end1 - start1   
    list_write.append(excute_time)
    list_write.append(excute_time1)
    writer.writerow(list_write)
    print("end down sampling")

    new_train_data = copy.deepcopy(train_set)
    list_write = []
    list_write.append("Adult")
    list_write.append("Massaging_opt")
    list_write.append(data.shape[0])
    list_write.append(train_set.shape[0])
    list_write.append(columns_compas[:len(columns_compas)-minus])
    list_write.append(len(columns_compas[:len(columns_compas)-minus]))
    excute_time = 0
    excute_time1 = 0
    print("start massaging")
    for a in all_names_lst:
      temp2, names = get_temp(new_train_data, all_names[a], compas_y)
      temp, temp_g = get_temp_g(new_train_data, names, compas_y)
      temp_g = temp_g[temp_g['cnt'] > filter_count]
      lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
      start = time.time()    
      need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
      end = time.time()
      excute_time += end - start

      new_train_data['skewed'] = 0
      new_train_data["diff"] = 0
      start1 = time.time()
      new_train_data = naive_massaging(new_train_data, temp2, names, need_pos, need_neg, compas_y)
      end1 = time.time()
      excute_time1 += end1 - start1   
    list_write.append(excute_time)
    list_write.append(excute_time1)
    writer.writerow(list_write)
    print("end massage sampling")

started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pref sampling
started pr

# NAIVE ALGORITHM

In [ ]:
import random
new_train_data = copy.deepcopy(train_set)
import csv

# get the list of numbers of the given group
def get_one_degree_neighbors(temp2, names, group_lst):
    result = []
    for i in range(len(group_lst)):
        d = copy.copy(temp2)
        for k in range(len(group_lst)):
            if k != i:
                d = d[d[names[k]] == group_lst[k]]
            else:
                d = d[d[names[k]] != group_lst[k]]
        result.append(d)
    return result

def determine_problematic(group_lst, temp2, result, label, threshold= 0.3):
    # return a value for a given group about whether it is a problematic group
    #0: ok group, 1: need negative records, 2: need positive records
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
 
    neighbors = compute_neighbors(group_lst, result)
    if(neighbors[2] == -1):
        # there is no neighbors
        return 0
    if(total > 10):
        # need to be large enough
        if (pos/(neg+1) - neighbors[2] > threshold):
            # too many positive records
            return 1
        if (neighbors[2] - pos/(neg+1) > threshold):
            # too many negative records
            return 2
    return 0

with open('adult_records.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    minus = 0
    # get groups for the right number of attributes
    all_names = candidate_groups(columns_compas[:len(columns_compas)- minus])
    all_names_lst = list(all_names.keys())[len(columns_compas)-minus+1:]
    all_names_lst.reverse()
    
    # add all of the settings to a list to write to results csv 
    list_write = []
    list_write.append("Adult")
    list_write.append("Pref_naive")
    list_write.append(data.shape[0])
    list_write.append(train_set.shape[0])
    list_write.append(columns_compas[:len(columns_compas)-minus])
    list_write.append(len(columns_compas[:len(columns_compas)-minus]))
    excute_time = 0
    excute_time1 = 0
    for a in all_names_lst:
      need_pos = []
      need_neg = []
      temp2, names = get_temp(new_train_data, all_names[a], compas_y)
      temp, temp_g = get_temp_g(new_train_data, names, compas_y)
      # naive algorithm do not have a filter here.
      # temp_g = temp_g[temp_g['cnt'] > filter_count]
      # start finding the set of problematic regions
      start = time.time()
      for index, row in temp_g.iterrows():
        group_lst = []
        for n in names:
          group_lst.append(row[n])
        # get neighbors 
        result = get_one_degree_neighbors(temp2, names, group_lst)
        # get if belong in need/pos group
        pos_neg_val = determine_problematic(group_lst, temp2, result, compas_y, threshold= 0.3)
        # if needs neg vals 
        if pos_neg_val == 1:
          need_neg.append(group_lst)
      # if needs pos vals
        if pos_neg_val == 2:
          need_pos.append(group_lst)
      end = time.time()
      excute_time += end - start
        
    # if the naive algorithm is taking too long, break at 200 secs (timeout)
      if excute_time > 200:
        break
      new_train_data['skewed'] = 0
      new_train_data["diff"] = 0  
      # call pref sampling 
      start1 = time.time()    
      new_train_data = pref_sampling_opt(new_train_data, names, compas_y, need_pos, need_neg)
      end1 = time.time()
      excute_time1 += end1 - start1   

    list_write.append(excute_time)
    list_write.append(excute_time1)
    writer.writerow(list_write)